In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
#nltk.download('stopwords')

#read in text file
with open('spam.txt','r', encoding='utf8') as f:
    words=f.read().split()
    words=[x.lower() for x in words]

#count number of occurences
d={}
stop_words=set(stopwords.words('english'))
for w in words:
    if w not in stop_words:
        if w in d:
            d[w]+=1
        else:
            d[w]=1
#print top 10            
lst_spam=sorted([(d[w],w) for w in d], reverse=True)
print("Top 10 words are /n")
i=1
for count, word in lst_spam[:10]:
    print('%2s. %4s %s' %(i, count, word))
    i+=1

Top 10 words are /n
 1. 2087 number
 2.  319 url
 3.  245 hyperlink
 4.  187 free
 5.  143 click
 6.  114 email
 7.  111 please
 8.  109 list
 9.   93 get
10.   86 name


In [2]:
#read in text file
with open('not_spam.txt','r', encoding='utf8') as f:
    words=f.read().split()
    words=[x.lower() for x in words]

    
#count number of occurences   
d={}
stop_words=set(stopwords.words('english'))
for w in words:
    if w not in stop_words:
        if w in d:
            d[w]+=1
        else:
            d[w]=1
#print top 10            
lst_not_spam=sorted([(d[w],w) for w in d], reverse=True)
print("Top 10 words are /n")
i=1
for count, word in lst_not_spam[:10]:
    print('%2s. %4s %s' %(i, count, word))
    i+=1

Top 10 words are /n
 1. 5521 number
 2. 2122 url
 3.  538 one
 4.  523 people
 5.  461 would
 6.  440 like
 7.  422 get
 8.  408 wrote
 9.  405 said
10.  378 list


In [3]:
#grab top 10 words in both lists
lst_spam=lst_spam[:10]
lst_not_spam=lst_not_spam[:10]
lst_spam=[item[1] for item in lst_spam]
lst_not_spam=[item[1] for item in lst_not_spam]
#grab words in lst_spam that are not in lst_not_spam
list_final=np.setdiff1d(lst_spam,lst_not_spam)
list_final

array(['click', 'email', 'free', 'hyperlink', 'name', 'please'],
      dtype='<U9')

In [4]:
#create list of words that are differentiators
text_list=['click','email','free','hyperlink','name''please']

In [5]:
#read in data
df_spam = pd.read_csv("spam_or_not_spam.csv")
#keep only rows with email data
df_spam=df_spam[df_spam['email'].notna()]
#create lowercase email col
df_spam['email lower']=df_spam['email'].str.lower()
df_spam.head()

,email,label,email lower
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0,date wed number aug number number number numb...
1,martin a posted tassos papadopoulos the greek ...,0,martin a posted tassos papadopoulos the greek ...
2,man threatens explosion in moscow thursday aug...,0,man threatens explosion in moscow thursday aug...
3,klez the virus that won t die already the most...,0,klez the virus that won t die already the most...
4,in adding cream to spaghetti carbonara which ...,0,in adding cream to spaghetti carbonara which ...


In [6]:
#create unique lowercase email col
def drop_duplicates(row):
    #split strings by comma, drop duplicates, and join back
    words=row.split(' ')
    return ' '.join(np.unique(words).tolist())
df_spam['email lower unique']=df_spam['email lower'].apply(drop_duplicates)
df_spam.head()

,email,label,email lower,email lower unique
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0,date wed number aug number number number numb...,_____________________________________________...
1,martin a posted tassos papadopoulos the greek ...,0,martin a posted tassos papadopoulos the greek ...,a admiring alexander amphitheatre an and are ...
2,man threatens explosion in moscow thursday aug...,0,man threatens explosion in moscow thursday aug...,a about agencies an and ap appeared armed att...
3,klez the virus that won t die already the most...,0,klez the virus that won t die already the most...,_____________________________________________...
4,in adding cream to spaghetti carbonara which ...,0,in adding cream to spaghetti carbonara which ...,a adding an and are as ask authentic because ...


In [7]:
#find count of matches
pat='|'.join(r"\b{}\b".format(x) for x in text_list)
df_match=df_spam[df_spam['email lower'].str.contains(pat)].copy()
df_match['unique count of matches']=df_spam['email lower unique'].str.count(pat)
#calculate percent match
df_match['percent match']=(df_match['unique count of matches']/6)*100
df_match.head()

,email,label,email lower,email lower unique,unique count of matches,percent match
1,martin a posted tassos papadopoulos the greek ...,0,martin a posted tassos papadopoulos the greek ...,a admiring alexander amphitheatre an and are ...,2,33.333333
2,man threatens explosion in moscow thursday aug...,0,man threatens explosion in moscow thursday aug...,a about agencies an and ap appeared armed att...,2,33.333333
4,in adding cream to spaghetti carbonara which ...,0,in adding cream to spaghetti carbonara which ...,a adding an and are as ask authentic because ...,2,33.333333
5,i just had to jump in here as carbonara is on...,0,i just had to jump in here as carbonara is on...,a an and are as ask bacon basically beaten ca...,2,33.333333
6,the scotsman NUMBER august NUMBER playboy want...,0,the scotsman number august number playboy want...,a age ageing all an and applications arms att...,2,33.333333


In [8]:
#keep only text and label cols
df_spam_model=pd.read_csv("spam_or_not_spam_model.csv")
df_spam_model.columns = ['text', 'label']
df_spam_model.head()

,text,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [9]:
#create a text list
text = list(df_spam_model['text'])

#preprocessing loop
import re
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

corpus = []

#remove special characters, convert to lowercase, split words (tokenize), ignore stopwords, reduce to base form (lemmatize)
for i in range(len(text)):
    r = re.sub('[^a-zA-Z]', ' ', text[i])
    r = r.lower()
    r = r.split()
    r = [word for word in r if word not in stopwords.words('english')]
    r = [lemmatizer.lemmatize(word) for word in r]
    r = ' '.join(r)
    corpus.append(r)

#assign corpus to text col
df_spam_model['text'] = corpus

df_spam_model

,text,label
0,date wed number aug number number number numbe...,0
1,martin posted tasso papadopoulos greek sculpto...,0
2,man threatens explosion moscow thursday august...,0
3,klez virus die already prolific virus ever kle...,0
4,adding cream spaghetti carbonara effect pasta ...,0
...,...,...
95,attention must computer user new special packa...,1
96,interest rate cut perfect time think refinanci...,1
97,miss amazing business opportunity weight loss ...,1
98,friend brand new idea live webcam hyperlink cl...,1


In [10]:
#create feature and label sets
X = df_spam_model['text']
y = df_spam_model['label']

#train test split (66% train - 33% test)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

print('Training Data :', X_train.shape)
print('Testing Data : ', X_test.shape)

Training Data : (67,)
Testing Data :  (33,)


In [11]:
#train bag of words model (feature extraction)
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
X_train_cv = cv.fit_transform(X_train)
X_train_cv.shape

(67, 2325)

In [12]:
#train logistic regression model
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train_cv, y_train)

#transform X_test using CV
X_test_cv = cv.transform(X_test)

#generate predictions
predictions = lr.predict(X_test_cv)
predictions

array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0], dtype=int64)

In [13]:
#confusion matrix
from sklearn import metrics
#default scikitlearn order is [negative, positive], labels can change that (doesn't matter in this case because same as default)
df_matrix = pd.DataFrame(metrics.confusion_matrix(y_test,predictions,labels=[0,1]),index=['true not spam','true spam'], columns=['pred not spam','pred spam'])
df_matrix

#to apply to dataset just repeat steps starting from create a text list

,pred not spam,pred spam
true not spam,15,2
true spam,0,16
